In [1]:
import pymongo
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import time

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#my read books (page 1)
#note: 30 books per page by default (?)
url = 'https://www.goodreads.com/review/list/8178894?shelf=read&page=1'

In [3]:
r = requests.get(url)
html = r.content

In [17]:
soup = BeautifulSoup(html, 'html.parser')

In [18]:
print(soup.prettify())

<!DOCTYPE html>
<html class="desktop">
 <head>
  <title>
   Nostalgebraist's bookshelf: read (showing 1-30 of 257)

(sorted by: date added)
  </title>
  <script type="text/javascript">
   var ue_t0=window.ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElement

In [19]:
stars = soup.select('.staticStars')

In [20]:
len(stars)

30

In [27]:
stars[0]

<span class=" staticStars"><span class="staticStar p10" size="15x15" title="really liked it">really liked it</span><span class="staticStar p10" size="15x15" title="really liked it"></span><span class="staticStar p10" size="15x15" title="really liked it"></span><span class="staticStar p10" size="15x15" title="really liked it"></span><span class="staticStar p0" size="15x15" title="really liked it"></span></span>

In [28]:
len(stars[0].select('.staticStar.p10'))

4

In [29]:
myRatingStars = soup.select('.stars')

In [30]:
len(myRatingStars)

30

In [35]:
int(myRatingStars[0]['data-resource-id'])

22590

In [36]:
#OK, now let's put that all together (for page 1)

In [37]:
stars = soup.select('.staticStars')
myRatingStars = soup.select('.stars')

bookIDs = []
bookRatings = []

for userRating, myRating in zip(stars, myRatingStars):
    bookIDs.append(int(myRating['data-resource-id']))
    bookRatings.append(len(userRating.select('.staticStar.p10')))

In [43]:
np.array([bookIDs, bookRatings]).T

array([[   22590,        4],
       [   55563,        4],
       [ 2074138,        2],
       [26861260,        5],
       [17347389,        3],
       [   80268,        4],
       [15995747,        4],
       [   71845,        2],
       [ 1136307,        4],
       [  400354,        3],
       [   42143,        3],
       [   28921,        5],
       [13538873,        3],
       [   27578,        3],
       [28253018,        4],
       [   17690,        2],
       [11173188,        3],
       [   64216,        4],
       [  663648,        4],
       [ 1401403,        5],
       [  612143,        4],
       [  567657,        4],
       [11173187,        4],
       [23995360,        5],
       [ 1079601,        3],
       [ 2169648,        3],
       [23363928,        5],
       [18265940,        4],
       [10100051,        4],
       [18769508,        3]])

In [44]:
#now for all pages

In [5]:
bookIDs = []
bookRatings = []

url = 'https://www.goodreads.com/review/list/8178894?shelf=read&page=1'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

curShelfString = soup.select('.selectedShelf')[0].get_text()
numBooksOnCurShelf = int(curShelfString[curShelfString.rfind('(')+  1 : curShelfString.rfind(')')])
numPages = numBooksOnCurShelf/30 + 1

for i in range(1,numPages+1):
    url = 'https://www.goodreads.com/review/list/8178894?shelf=read&page=' + str(i)
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    stars = soup.select('.staticStars')
    myRatingStars = soup.select('.stars')

    for userRating, myRating in zip(stars, myRatingStars):
        bookIDs.append(int(myRating['data-resource-id']))
        bookRatings.append(len(userRating.select('.staticStar.p10')))
    
    time.sleep(1)

userRatingArray = np.array([bookIDs, bookRatings]).T

In [6]:
userRatingArray = np.array([bookIDs, bookRatings]).T

In [7]:
userRatingDf = pd.DataFrame(userRatingArray, columns=['bookID','rating'])
userRatingDf['rating'].describe()

count    257.000000
mean       3.182879
std        1.136065
min        0.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: rating, dtype: float64